In [1]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os

In [2]:
places

{'starbucks': {'color': 'blue',
  'icon': 'coffee',
  'rank': 3,
  'distance': 1000,
  'api': 'Starbucks coffee'},
 'airport': {'color': 'red',
  'icon': 'plane',
  'rank': 20,
  'distance': 20000,
  'api': 'Airport'},
 'school': {'color': 'yellow',
  'icon': 'pen',
  'rank': 10,
  'distance': 1500,
  'api': 'Primary School'},
 'party': {'color': 'green',
  'icon': 'glass',
  'rank': 5,
  'distance': 1000,
  'api': 'party'}}

In [3]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [5]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }
}
) 

In [6]:
data_companies = pd.DataFrame(companies)

In [7]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [8]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 5000
list_number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    list_number_offices.append(len(num_offices))
data_companies['number offices near'] = list_number_offices

In [9]:
data_companies = data_companies[data_companies['number offices near'] > 1]

In [10]:
data_companies.head()

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number offices near
0,5e049e9de6eb90cad2d707ad,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
2,5e049e9de6eb90cad2d707af,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
3,5e049e9de6eb90cad2d707b0,Skydeck,9.0,2012,web,52cdef7c4bab8bd675298509,4000000.0,Palo Alto,USA,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2
4,5e049e9de6eb90cad2d707b1,PeekYou,20.0,2012,web,52cdef7c4bab8bd675297f94,1830000.0,New York,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",7
9,5e049e9de6eb90cad2d707b7,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",2


In [11]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [12]:
def getNearby(lat,lng,query,radius):
        CLIENT_ID = os.getenv("client_id")
        CLIENT_SECRET = os.getenv("client_secret")
        url = 'https://api.foursquare.com/v2/venues/explore'
        queryParams = dict(
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            v='20181023', # DONDE ESTOY
            ll=f"{lat},{lng}", #QUE QUIERO
            query=query, # A QUE DISTANCIA
            radius=radius
        )
        res = requests.get(url, params=queryParams)
        return res.json()
  

In [13]:
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
        except:
            Exception: "Ha habido problemas con este {}".formate(place)
        

In [14]:
request

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5e04f3921a4b0a0028cdbe5a'},
 'response': {}}

In [20]:
def curateResponse(data):
    places = []
    for item in data["response"]["groups"][0]["items"]:
        places.append({
            "type": item["venue"]['categories'][0]['name'],
            "name":item["venue"]['name'],
            "address":item["venue"]['location'],
            "distance": item["venue"]['location']["distance"],
            "coordinates": {
            "type":"Point",
            "coordinates":[item["venue"]['location']["lng"], item["venue"]['location']["lat"]]
                }})
    return places

In [21]:
df = pd.DataFrame(curateResponse(request))

In [22]:
df["type"].unique()

array(['Nightclub', 'French Restaurant', 'Roof Deck', 'Hotel',
       'Chinese Restaurant', 'Bar', 'Hotel Bar', 'Asian Restaurant',
       'Cocktail Bar', 'New American Restaurant', 'Burger Joint',
       'Spanish Restaurant', 'Club House', 'Steakhouse', 'Grocery Store',
       'Italian Restaurant', 'Wine Bar', 'Gay Bar', 'Seafood Restaurant',
       'Tea Room'], dtype=object)